In [47]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression, LassoCV, RidgeCV

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import SVG
#from graphviz import Source

## Building the initial models

In [123]:
data = pd.read_csv("data/taxi_FINAL.csv")

In [106]:
data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'duration',
       'Location', 'time_of_day', 'CRD', 'UNK', 'NOC', 'DIS', 'CSH',
       'tip_amount_bin'],
      dtype='object')

In [124]:
data.corr()["tip_amount"]

vendor_id           -0.031161
passenger_count     -0.003495
trip_distance        0.551027
pickup_longitude    -0.003495
pickup_latitude     -0.011209
dropoff_longitude   -0.015585
dropoff_latitude    -0.006701
fare_amount          0.624147
surcharge            0.048372
mta_tax                   NaN
tip_amount           1.000000
tolls_amount        -0.000341
total_amount         0.743195
duration             0.569341
Location            -0.020564
time_of_day          0.025375
CRD                  0.085818
UNK                 -0.008303
NOC                 -0.119688
DIS                 -0.063635
CSH                  0.001348
Name: tip_amount, dtype: float64

In [149]:
x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['tip_amount','pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude','pickup_datetime','dropoff_datetime','total_amount','mta_tax']), data['tip_amount'], test_size=0.30, random_state=1)

In [126]:
data.corr()["tip_amount"]

vendor_id           -0.031161
passenger_count     -0.003495
trip_distance        0.551027
pickup_longitude    -0.003495
pickup_latitude     -0.011209
dropoff_longitude   -0.015585
dropoff_latitude    -0.006701
fare_amount          0.624147
surcharge            0.048372
mta_tax                   NaN
tip_amount           1.000000
tolls_amount        -0.000341
total_amount         0.743195
duration             0.569341
Location            -0.020564
time_of_day          0.025375
CRD                  0.085818
UNK                 -0.008303
NOC                 -0.119688
DIS                 -0.063635
CSH                  0.001348
Name: tip_amount, dtype: float64

In [127]:
linreg = LinearRegression()
linreg.fit(x_train, y_train)

train_score = linreg.score(x_train, y_train)
test_score = linreg.score(x_test, y_test)
print('The train score is {} and the test score is {}'.format(train_score, test_score))
print('\n')
betas = pd.Series(linreg.coef_, index=x_train.columns)
betas = betas.append(pd.Series({"Intercept": linreg.intercept_}))
print(betas)

The train score is 0.40563019790279475 and the test score is 0.4058872085786299


vendor_id         -0.067295
passenger_count    0.000405
trip_distance      0.009393
fare_amount        0.137949
surcharge          0.130248
tolls_amount       0.717954
duration           0.000092
Location          -0.000245
time_of_day        0.003297
CRD                0.565141
UNK                0.484054
NOC               -0.788367
DIS               -0.905965
CSH                0.645137
Intercept         -0.073568
dtype: float64


In [128]:
# We also save the model results
all_models = {}
all_models['linreg'] = (train_score, test_score, linreg, 'runs on x_train')
all_models

{'linreg': (0.40563019790279475,
  0.4058872085786299,
  LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
           normalize=False),
  'runs on x_train')}

In [129]:
from sklearn import metrics

cart_model = DecisionTreeRegressor(max_depth=5, random_state=1)
cart_model.fit(x_train, y_train)
y_pred= cart_model.predict(x_test)

train_score = cart_model.score(x_train, y_train)
test_score = cart_model.score(x_test, y_test)

#pre = precision_score(y_test, y_pred) # since the dataset is imbalanced
#rec = recall_score(y_test, y_pred)

#print('On the validation set, the precision is {} and recall is {}'.format(pre, rec))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('The train score is {} and the test score is {}'.format(train_score, test_score))

Mean Absolute Error: 0.45355278300531926
Mean Squared Error: 0.4167062107945469
Root Mean Squared Error: 0.6455278543909216
The train score is 0.40171923408195875 and the test score is 0.4018854857377855


In [130]:
all_models['cart_model'] = (train_score, test_score, cart_model, 'runs on x_train')

In [131]:
# Fit the model
lasso = LassoCV()
lasso.fit(x_train, y_train)

# calculate the score
train_score = lasso.score(x_train, y_train)
test_score = lasso.score(x_test, y_test)

print('The train score is {} and the test score is {}'.format(train_score, test_score)) 

# calculate betas
print('\n')
betas = pd.Series(lasso.coef_, index= x_train.columns)
betas = betas.append(pd.Series({"Intercept": lasso.intercept_}))
print(betas)
              

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


The train score is 0.37276915209054406 and the test score is 0.37333905182913296


vendor_id         -0.000000
passenger_count   -0.000000
trip_distance      0.000000
fare_amount        0.073218
surcharge          0.000000
tolls_amount      -0.000000
duration           0.000726
Location           0.000000
time_of_day        0.000000
CRD                0.000000
UNK               -0.000000
NOC               -0.000000
DIS               -0.000000
CSH               -0.000000
Intercept          0.651679
dtype: float64


In [132]:
all_models['lasso'] = (train_score, test_score, lasso, 'runs on x_train')

In [ ]:
# Visualize the decision tree
cart_graph = Source(export_graphviz(cart_model,
                   feature_names = df.columns[1:],
                   rounded = True, proportion = False, 
                  filled = True))
SVG(cart_graph.pipe(format='svg'))

In [133]:
from sklearn.ensemble import RandomForestRegressor

# Train the CART model
rfc_model = RandomForestRegressor(max_depth=5,random_state=1)
rfc_model.fit(x_train, y_train)
y_pred= rfc_model.predict(x_test)

train_score = rfc_model.score(x_train, y_train)
test_score = rfc_model.score(x_test, y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('The train score is {} and the test score is {}'.format(train_score, test_score)) 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Mean Absolute Error: 0.4532240792796501
Mean Squared Error: 0.41638532020391744
Root Mean Squared Error: 0.6452792575342226
The train score is 0.4021872357624784 and the test score is 0.4023460723927807


In [134]:
all_models['rfc'] = (train_score, test_score, rfc_model, 'runs on x_train')

## Feature Engineering

In [135]:
data.columns

Index(['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count',
       'trip_distance', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'duration',
       'Location', 'time_of_day', 'CRD', 'UNK', 'NOC', 'DIS', 'CSH'],
      dtype='object')

In [136]:
data.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,...,tolls_amount,total_amount,duration,Location,time_of_day,CRD,UNK,NOC,DIS,CSH
0,1,2014-01-09 20:45:25,2014-01-09 20:52:31,1,0.7,-73.994770,40.736828,-73.982227,40.731790,6.5,...,0.0,8.90,426,5,4,1,0,0,0,0
1,1,2014-01-09 20:46:12,2014-01-09 20:55:12,1,1.4,-73.982392,40.773382,-73.960449,40.763995,8.5,...,0.0,11.40,540,4,4,1,0,0,0,0
2,1,2014-01-09 20:44:47,2014-01-09 20:59:46,2,2.3,-73.988570,40.739406,-73.986626,40.765217,11.5,...,0.0,14.00,899,5,4,1,0,0,0,0
3,1,2014-01-09 20:44:57,2014-01-09 20:51:40,1,1.7,-73.960213,40.770464,-73.979863,40.777050,7.5,...,0.0,10.20,403,5,4,1,0,0,0,0
4,1,2014-01-09 20:47:09,2014-01-09 20:53:32,1,0.9,-73.995371,40.717248,-73.984367,40.720524,6.0,...,0.0,8.75,383,5,4,1,0,0,0,0


In [137]:
cont = ["passenger_count","tip_distance","fare_amount","surcharge","tolls_amount","duration"]

In [145]:
base_corrs = data.corr().tip_amount.drop(index=['tip_amount','mta_tax',"pickup_longitude",
                        "pickup_latitude","dropoff_longitude","dropoff_latitude","total_amount"])
base_corrs

vendor_id         -0.031161
passenger_count   -0.003495
trip_distance      0.551027
fare_amount        0.624147
surcharge          0.048372
tolls_amount      -0.000341
duration           0.569341
Location          -0.020564
time_of_day        0.025375
CRD                0.085818
UNK               -0.008303
NOC               -0.119688
DIS               -0.063635
CSH                0.001348
Name: tip_amount, dtype: float64

In [154]:
for feature1 in base_corrs.index:
    for feature2 in base_corrs.index:
        if feature2 != feature1:
            # divide the two features to create a new feature
            new_feature = x_train[feature1] / x_train[feature2]
            new_corr = np.abs(np.round(np.corrcoef(new_feature, y_train)[0,1], 3))
            corr1 = np.abs(np.round(base_corrs[feature1], 3))
            corr2 = np.abs(np.round(base_corrs[feature2], 3))
            # add a threshold of 0.02 to make sure that the improvement is meaningful
            if new_corr > max(corr1, corr2)+0.015:
                print('{} {} combine to get correlation {} compared to {} {}'.format(
                    feature1, feature2, new_corr, corr1, corr2))

vendor_id time_of_day combine to get correlation 0.047 compared to 0.031 0.025
Location time_of_day combine to get correlation 0.044 compared to 0.021 0.025


In [155]:
for feature1 in base_corrs.index:
    for feature2 in base_corrs.index:
        if feature2 != feature1:
            # divide the two features to create a new feature
            new_feature = x_train[feature1] * x_train[feature2]
            new_corr = np.abs(np.round(np.corrcoef(new_feature, y_train)[0,1], 3))
            corr1 = np.abs(np.round(base_corrs[feature1], 3))
            corr2 = np.abs(np.round(base_corrs[feature2], 3))
            # add a threshold of 0.02 to make sure that the improvement is meaningful
            if new_corr > max(corr1, corr2)+0.015:
                print('{} {} combine to get correlation {} compared to {} {}'.format(
                    feature1, feature2, new_corr, corr1, corr2))